In [1]:
import requests as req
import json
import urllib.parse
import pandas as pd
import io
import ast
from bs4 import BeautifulSoup

In [2]:
def entitify(text):
    return urllib.parse.quote(text)

In [3]:
def fwrite(response, fname):
    file = open(fname, "w")

    file.write(response.text)

    file.close()

In [4]:
def demonstrate_json(res):
    print(json.dumps(res, indent=4, sort_keys=True, ensure_ascii = False))

In [5]:
def get_client_list(q):
    
    headers = {'Accept': 'application/json, text/plain, */*',
              'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
               }
    
    
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/search/interactive/composite?searchValue=' 

    q = entitify(q)
    
    url = url + q
    
    r = req.get(url, headers=headers)
    
    res = r.json()
    
    return res
    



In [6]:
#страница клиента по списку клиентов, типу клиента и названию

def get_client_page(res, TITLE, NAME):
    
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%B5%D0%B4%D0%B8%D0%BD%D1%8B%D0%B9-%D1%80%D0%B5%D0%B5%D1%81%D1%82%D1%80-%D0%B7%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%89%D0%B8%D0%BA%D0%BE%D0%B2/'

    headers = {'Accept': 'application/json, text/plain, */*',
              'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
               }
    
    header2 = 'title'
    
    header3 = 'searchValue'

    data = json.dumps(res, indent=4, sort_keys=True)

    df = pd.DataFrame(ast.literal_eval(data)['data'])
    
    
    row = df[(df[header2] == TITLE) & (df[header3] == NAME)]
    
    cardId = row['cardId'].iloc[0]
    
    title = entitify(TITLE.lower())

    url = url + title + '/' + cardId
    
    r = req.get(url, headers=headers)
     
    return r.text

In [7]:
def get_doquments_archive(cardId):
    
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/erz/deverz/'+cardId+'/document/download?tab=projectDeclarations&tab=developerReport'

    with req.get(url, stream=True) as r:
        r.raise_for_status()
        with open(cardId + '.zip', 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

In [8]:
def get_proj_list(search, cardId):
    
    headers = {'Accept': 'application/json, text/plain, */*',
              'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
               }
    
    search = entitify(search)
    
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%BA%D0%B0%D1%82%D0%B0%D0%BB%D0%BE%D0%B3-%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D1%80%D0%BE%D0%B5%D0%BA/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA-%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2/%D1%81%D0%BF%D0%B8%D1%81%D0%BE%D0%BA?search='
    url1 = '&devId='
    url2 = '&status=buildingFlag'

    r = req.get(url + search + url1 + cardId + url2, headers=headers).text
    
    parsed_html = BeautifulSoup(r, 'html.parser')
    data = json.loads(parsed_html.find('script', id="__NEXT_DATA__", type='application/json').text)
    return data
    
    #res = r.json()
    #return res
    
    #demonstrate_json(res)
    
    
    
    

In [9]:
def parse_client_page(client_page):
    res = client_page
    parsed_html = BeautifulSoup(res, 'html.parser')

    data = json.loads(parsed_html.find('script', id="__NEXT_DATA__", type='application/json').text)


    banks = data["props"]['initialProps']["pageProps"]['banks']

    beneficiaries = data["props"]['initialProps']["pageProps"]['beneficiaries']

    documents = data["props"]['initialProps']["pageProps"]["documents"]

    reports = data["props"]['initialProps']["pageProps"]["documents"]['report']

    reports = data["props"]['initialProps']["pageProps"]["documents"]['report']

    rpd = data["props"]['initialProps']["pageProps"]["documents"]['rpd']

In [10]:
span class="_227lI"

li class="_2ag2R"

ul class="_3U-mA _23Irb"



SyntaxError: invalid syntax (<ipython-input-10-52e1d84f08be>, line 1)

In [16]:
#res = get_client_list('строй тэк')
#res
#res = get_client_page(res, 'Застройщик', 'СЗ МИЦ-МИЦ')

#parse_client_page(res)



In [29]:
list_of_inns = []
with open('list_of_inns.txt', 'r') as f:
    list_of_inns.append(f.read().split('\n\n'))
list_of_inns = list_of_inns[0]

In [31]:
def get_client_page_by_inn(inn):
    
    cardId = get_client_list(inn)['data'][0]['cardId']
    
    TITLE = 'застройщик'
    
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/%D0%B5%D0%B4%D0%B8%D0%BD%D1%8B%D0%B9-%D1%80%D0%B5%D0%B5%D1%81%D1%82%D1%80-%D0%B7%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D1%89%D0%B8%D0%BA%D0%BE%D0%B2/'

    headers = {'Accept': 'application/json, text/plain, */*',
              'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7' 
               }
    
    title = entitify(TITLE.lower())
    
    url = url + title + '/' + cardId
    
    r = req.get(url, headers=headers)
     
    return r.text
    
                    


    

In [32]:
for inn in list_of_inns:
    get_client_page_by_inn(inn)

IndexError: list index out of range

In [33]:
def get_archives_set(list_of_inns):
    
    for inn in list_of_inns:
        
        res = get_client_page_by_inn(inn)
        
        parse_client_page(res)